In [1]:
import os
import shutil
from creds import OPENAI_KEY
from application_generator import JobApplicationBuilder
from utils import LatexToolBox, text_to_pdf
from models import ChatGPT
from prompts.resume_section_prompts import RESUME_WRITER_PERSONA

llm = ChatGPT(
    api_key=OPENAI_KEY,
    model="gpt-4o",
    system_prompt=RESUME_WRITER_PERSONA,
    max_output_tokens=None,
    temperature=0.7
)

generator = JobApplicationBuilder(
    llm=llm,
)

job_content_path = r"input_data/R149561 Data Scientist in Seattle San FRan.pdf"
resume_pdf_path = r"input_data/full_resume_contents_20241124.pdf"
#resume_pdf_path = r"input_data/Grad School Resume.pdf"
job_details_dict, job_details_filepath = generator.extract_job_content(job_content_path)
resume_dict = generator.user_data_extraction(user_data_path=resume_pdf_path)
resume_details_dict, resume_details_filepath = generator.generate_resume_json(job_content=job_details_dict,
                                                                              user_data=resume_dict)

resume_latex, resume_tex_path = generator.resume_json_to_resume_tex(resume_details=resume_details_dict)
print("Done generating resume tex file")

cover_letter, cover_letter_txt_path = generator.generate_cover_letter(job_details=job_details_dict,
                                                                      user_data=resume_details_dict,
                                                                      need_pdf=False)

# check that the fonts in the resume tex are available and flag the ones not installed
font_statuses = LatexToolBox.check_fonts_installed(resume_tex_path)
if not all([v for k, v in font_statuses.items()]):
    [print(f"{k} not installed") for k, v in font_statuses.items() if not v]

/Users/aaronrdankert/projects/job_hunter_toolbox/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Job Details JSON generated at: output_files/Adobe/DataScienceEngi_20241220172036/JD.json

Fetching user data...


/Users/aaronrdankert/projects/job_hunter_toolbox/.venv/lib/python3.11/site-packages/pydantic/json_schema.py:2191: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='The name or title of the project.'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/Users/aaronrdankert/projects/job_hunter_toolbox/.venv/lib/python3.11/site-packages/pydantic/json_schema.py:2191: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='The type or category of the project, such as hackathon, publication, professional, and academic.'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/Users/aaronrdankert/projects/job_hunter_toolbox/.venv/lib/python3.11/site-packages/pydantic/json_schema.py:2191: PydanticJsonSchemaWarning: Default va


Generating Resume Details...
Processing Resume's WORK_EXPERIENCE Section...
Processing Resume's PROJECTS Section...
Processing Resume's SKILL_SECTION Section...
Processing Resume's EDUCATION Section...
Processing Resume's CERTIFICATIONS Section...
Processing Resume's ACHIEVEMENTS Section...
Done generating resume tex file
Cover Letter generated at:  output_files/Adobe/DataScienceEngi_20241220172036/cover_letter.txt


Edit the resume latex before running the next cell.

In [2]:
LatexToolBox.compile_latex_to_pdf(tex_file_path=resume_tex_path)
print(f"Resume PDF is saved at {resume_tex_path.replace('.tex', '.pdf')}")

cover_letter_pdf_path = cover_letter_txt_path.replace('.txt', '.pdf')
cover_letter_pdf_path = text_to_pdf(cover_letter, cover_letter_pdf_path)
print(f"Cover Letter PDF is saved at {cover_letter_pdf_path}")

PDF successfully saved to /Users/aaronrdankert/projects/job_hunter_toolbox/output_files/Adobe/DataScienceEngi_20241220172036
Resume PDF is saved at output_files/Adobe/DataScienceEngi_20241220172036/resume.pdf
Cover Letter PDF is saved at output_files/Adobe/DataScienceEngi_20241220172036/cover_letter.pdf


In [3]:
# move the job content file to the output folder
shutil.move(job_content_path, generator.get_job_doc_path())

'output_files/Adobe/DataScienceEngi_20241220172036/_1'

In [ ]:
from utils import LatexToolBox
fonts_required, _ = LatexToolBox.extract_tex_font_dependencies('output_files/Adobe/InternMachineLe_20241209195126/resume.tex')
font_dict = LatexToolBox.check_fonts_installed(fonts_required)
print(font_dict)